# Implementando uma ESN com Grid Search para escolher melhor os parâmetros no cenário de Hénon

## 1. Importando as bibliotecas necessárias

### 1.1 Bibliotecas gerais

In [1]:
%matplotlib widget
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns 

sns.set_style("ticks")
plt.rcParams['savefig.dpi'] = 200
plt.rcParams["figure.dpi"] = 125

plt.rcParams.update({
    "text.usetex": True,
    "font.family": "serif",
    "font.serif": ["Palatino"],
})

### 1.2 Bibliotecas dos sistemas caóticos

In [2]:
from pibic2020.tools import timeseries
from pibic2020.data import henon

### 1.3 *Gridsearch* na ESN

In [3]:
from pibic2020.models import esn_model
from sklearn.model_selection import GridSearchCV
from sklearn.model_selection import TimeSeriesSplit 

## 2. Gerando a série temporal do mapa de Hénon

Os valores para os parâmetros das equações serão os "básicos" apresentados no *paper* original.

In [4]:
a = 1.4
b = 0.3
x_inicial = 1.0
y_inicial = 0.0
n_inicial = 0

estados = np.array([x_inicial, y_inicial, n_inicial])

mapa = henon.MapaHenon(estado_inicial=estados)

In [5]:
n_iteracoes = 5000
estados = mapa.simular(n_iteracoes)

In [6]:
x = estados[:, 0]
y = estados[:, 1]
n = estados[:, 2]

### 2.1 Dividindo em um conjunto de treinamento e de teste, para K = 4 e L = 3

In [7]:
K = 4
L = 3
tam_teste = 0.15

In [8]:
serie_temporal = timeseries.SerieTemporal(x, K=K, L=L)

In [9]:
X_treino, X_teste, y_treino, y_teste = serie_temporal.dividir_treino_teste(tam_teste)

## 3. Definindo o modelo para a ESN e executando os Gridsearches

In [10]:
holdout = TimeSeriesSplit(n_splits=4, test_size=int(0.1*len(y_treino)), gap=0)

### 3.1 Criando o objeto da ESN

In [11]:
esn = esn_model.ModeloESN()

Utilizaremos os seguintes parâmetros no *Grid Search*:

In [12]:
spectral_radius = [[0.1], [0.2], [0.3], [0.4], [0.5],
                   [0.6], [0.7], [0.8], [0.9], [0.95],
                   [0.96], [0.97], [0.98], [0.99], [1]]

n_neurons = [30, 50, 70, 90, 100, 120, 140, 160, 180, 200, 240, 280, 320, 360, 400, 440, 480, 500]

Para facilitar, dividiremos esse processo em etapas.

### 3.2 Definindo e executando o primeiro *Grid Search*

Primeiro, avaliaremos o conjunto de *n_neurons* com o *spectral_radius*.

In [15]:
param_grid_1 = dict(spectral_radius=spectral_radius,
                    n_neurons=n_neurons)

In [16]:
grid = GridSearchCV(estimator=esn, param_grid=param_grid_1, n_jobs=-1, cv=holdout, scoring='neg_mean_squared_error', verbose=1)

In [17]:
grid_result = grid.fit(X_treino, y_treino)

Fitting 4 folds for each of 270 candidates, totalling 1080 fits


In [18]:
print("Best: %f using %s" % (grid_result.best_score_, grid_result.best_params_))

Best: -0.000002 using {'n_neurons': 500, 'spectral_radius': [0.1]}


## Teste com o melhor modelo

In [20]:
esn = esn_model.ModeloESN(n_neurons=500, spectral_radius=[0.1])

In [21]:
esn.fit(X_treino, y_treino)

ModeloESN(n_neurons=500, spectral_radius=[0.1])

In [22]:
y_pred = esn.predict(X_teste)

In [26]:
fig, ax = plt.subplots()
ax.plot(n[len(n)-int(len(n)*tam_teste):,], y_teste, color='DarkGreen', label='Valor real', linewidth=0.9)
ax.plot(n[len(n)-int(len(n)*tam_teste):,], y_pred, color='DimGrey', label='ESN', linewidth=0.9)

ax.set_title("Comparação da predição da ESN com o valor real do cenário de Hénon\n utilizando a rede recorrente ótima no conjunto de teste")
ax.set_ylabel('$x(t)$')
ax.set_xlabel('$t$')
ax.set_xlim(4900, 5000)
    
ax.grid(True)
sns.despine()
ax.legend()

plt.show()

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …